In [1]:
import os, json, uuid, time, requests
from PIL import Image
from tqdm import tqdm

COMFY = "http://127.0.0.1:8188"
PROMPT_JSON = "image_qwen_image_edit.json"
OUTPUT_DIR = "results_qwen_api"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Upload image to ComfyUI ---
def upload_image(image_path, server=COMFY):
    img_name = os.path.basename(image_path)
    with open(image_path, "rb") as f:
        files = {"image": (img_name, f, "image/png")}
        data = {"type": "input", "overwrite": "true"}
        r = requests.post(f"{server}/upload/image", files=files, data=data, timeout=120)
        r.raise_for_status()
    return img_name


# --- Download result image ---
def download_image(filename, server=COMFY, *, subfolder="", folder_type="output"):
    params = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    r = requests.get(f"{server}/view", params=params, timeout=120)
    r.raise_for_status()
    local_path = os.path.join(OUTPUT_DIR, filename)
    with open(local_path, "wb") as f:
        f.write(r.content)
    return local_path


# --- Run ComfyUI prompt with adjustable parameters ---
def run_prompt(prompt_json_path, image_path, new_prompt, server=COMFY, seed=None, steps=None, cfg=None):
    with open(prompt_json_path, "r") as f:
        prompt_data = json.load(f)

    # Upload image
    uploaded_img = upload_image(image_path, server)
    print(f"✅ Uploaded: {uploaded_img}")

    # Patch LoadImage node
    if "78" in prompt_data:
        prompt_data["78"]["inputs"]["image"] = uploaded_img

    # Patch positive prompt
    if "76" in prompt_data:
        prompt_data["76"]["inputs"]["prompt"] = new_prompt

    # Clear negative prompt
    if "77" in prompt_data:
        prompt_data["77"]["inputs"]["prompt"] = ""

    # Patch KSampler parameters
    if "3" in prompt_data:
        ks = prompt_data["3"]["inputs"]
        if seed is not None:
            ks["seed"] = int(seed)
        if steps is not None:
            ks["steps"] = int(steps)
        if cfg is not None:
            ks["cfg"] = float(cfg)

    # Fix SaveImage node inputs
    if "60" in prompt_data:
        prompt_data["60"]["inputs"] = {
            "filename_prefix": "ComfyUI",
            "images": ["8", 0]
        }

    # Submit prompt
    client_id = str(uuid.uuid4())
    payload = {"prompt": prompt_data, "client_id": client_id}
    r = requests.post(f"{server}/prompt", json=payload, timeout=120)

    if r.status_code != 200:
        print("❌ ComfyUI rejected the request:")
        print("Status:", r.status_code)
        print("Response:", r.text)
        return None

    data = r.json()
    prompt_id = data.get("prompt_id", client_id)
    print(f"✅ Queued prompt: {prompt_id}")

    # Poll for output
    deadline = time.time() + 600
    while time.time() < deadline:
        hist = requests.get(f"{server}/history/{prompt_id}", timeout=60).json()
        item = hist.get(prompt_id)
        if item and "outputs" in item:
            for node_id, node_out in item["outputs"].items():
                for im in node_out.get("images", []):
                    fn = im.get("filename")
                    sub = im.get("subfolder", "")
                    typ = im.get("type", "output")
                    if fn:
                        out = download_image(fn, server, subfolder=sub, folder_type=typ)
                        print(f"✅ Saved: {out}")
                        return out
        time.sleep(0.5)

    print("❌ Timeout: No output image found.")
    return None


# --- Batch processor with tqdm progress ---
def process_folder(input_folder, output_folder, prompt_text, seed=None, steps=None, cfg=None):
    os.makedirs(output_folder, exist_ok=True)
    image_files = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".png", ".jpg", ".jpeg"))
    ]

    print(f"\n📂 Found {len(image_files)} image(s) to process.\n")
    success_count, skip_count, fail_count = 0, 0, 0

    for fname in tqdm(image_files, desc="Processing images", ncols=100):
        in_path = os.path.join(input_folder, fname)
        out_path = os.path.join(output_folder, fname)

        if os.path.exists(out_path):
            skip_count += 1
            tqdm.write(f"⚙️  Skipping {fname}, already exists.")
            continue

        try:
            tqdm.write(f"🚀 Processing {fname} ...")
            result = run_prompt(PROMPT_JSON, in_path, prompt_text, seed=seed, steps=steps, cfg=cfg)

            if result:
                orig_w, orig_h = Image.open(in_path).size
                out_img = Image.open(result)
                out_img = out_img.resize((orig_w, orig_h), Image.LANCZOS)
                out_img.save(out_path)
                success_count += 1
                tqdm.write(f"✅ Saved → {out_path} ({orig_w}x{orig_h})")
            else:
                fail_count += 1
                tqdm.write(f"⚠️ No output for {fname}")

        except Exception as e:
            fail_count += 1
            tqdm.write(f"❌ Error processing {fname}: {e}")

        time.sleep(1)

    print("\n📊 Summary:")
    print(f"✅ Success: {success_count}")
    print(f"⚙️  Skipped: {skip_count}")
    print(f"❌ Failed:  {fail_count}")
    print(f"📁 Output folder: {output_folder}")


# ------------------- Example Usage -------------------
if __name__ == "__main__":
    input_folder = "input/images"
    output_folder = "output/results_colorized_5_1.5_1"

    prompt_text = (
        "restore and colorize with vivid colors, "
        "colorize dresses with vivid colors, "
        "skin tones should be natural, realistic, "
        "and no warm or cool tint in the entire image"
    )

    process_folder(
        input_folder,
        output_folder,
        prompt_text,
        seed=123456789,   # 👈 your seed
        steps=5,         # 👈 number of denoising steps
        cfg=1.5           # 👈 CFG guidance
    )



📂 Found 18 image(s) to process.



Processing images:   0%|                                                     | 0/18 [00:00<?, ?it/s]

🚀 Processing input_8 (1).png ...
✅ Uploaded: input_8 (1).png
✅ Queued prompt: d1b15a99-e874-4910-8d10-d41d561d8164


Processing images:   0%|                                                     | 0/18 [10:00<?, ?it/s]

❌ Timeout: No output image found.
⚠️ No output for input_8 (1).png


...

🚀 Processing input_7 (1).png ...
✅ Uploaded: input_7 (1).png
✅ Queued prompt: 558a9872-2132-4a35-bb98-a911bdc238b4
✅ Saved: results_qwen_api/ComfyUI_00029_.png


Processing images:  94%|█████████████████████████████████████████▌  | 17/18 [36:57<01:31, 91.11s/it]

✅ Saved → output/results_colorized_5_1.5_1/input_7 (1).png (1087x829)


Processing images: 100%|███████████████████████████████████████████| 18/18 [36:58<00:00, 123.23s/it]



📊 Summary:
✅ Success: 17
⚙️  Skipped: 0
❌ Failed:  1
📁 Output folder: output/results_colorized_5_1.5_1


In [3]:
import os
import json
import uuid
import time
import requests
from PIL import Image

# ---------------- Configuration ----------------
COMFY = "http://127.0.0.1:8188"
PROMPT_JSON = "image_qwen_image_edit.json"
OUTPUT_DIR = "results_qwen_api"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ------------------------------------------------
def upload_image(image_path, server=COMFY):
    """Upload an image to ComfyUI server."""
    img_name = os.path.basename(image_path)
    with open(image_path, "rb") as f:
        files = {"image": (img_name, f, "image/png")}
        data = {"type": "input", "overwrite": "true"}
        r = requests.post(f"{server}/upload/image", files=files, data=data, timeout=120)
        r.raise_for_status()
    return img_name


def download_image(filename, server=COMFY, *, subfolder="", folder_type="output"):
    """Download generated image from ComfyUI output folder."""
    params = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    r = requests.get(f"{server}/view", params=params, timeout=120)
    r.raise_for_status()
    local_path = os.path.join(OUTPUT_DIR, filename)
    with open(local_path, "wb") as f:
        f.write(r.content)
    return local_path


def run_prompt(
    prompt_json_path,
    image_path,
    positive_prompt,
    negative_prompt="",
    server=COMFY,
    seed=None,
    steps=None,
    cfg=None
):
    """Run a single ComfyUI prompt with positive and negative text inputs."""
    with open(prompt_json_path, "r") as f:
        prompt_data = json.load(f)

    uploaded_img = upload_image(image_path, server)
    print(f"✅ Uploaded: {uploaded_img}")

    # LoadImage node
    if "78" in prompt_data:
        prompt_data["78"]["inputs"]["image"] = uploaded_img

    # Positive prompt
    if "76" in prompt_data:
        prompt_data["76"]["inputs"]["prompt"] = positive_prompt

    # Negative prompt
    if "77" in prompt_data:
        prompt_data["77"]["inputs"]["prompt"] = negative_prompt

    # Sampling options
    if "3" in prompt_data:
        ks = prompt_data["3"]["inputs"]
        if seed is not None:
            ks["seed"] = int(seed)
        if steps is not None:
            ks["steps"] = int(steps)
        if cfg is not None:
            ks["cfg"] = float(cfg)

    # SaveImage node fix
    if "60" in prompt_data:
        prompt_data["60"]["inputs"] = {
            "filename_prefix": "ComfyUI",
            "images": ["8", 0]
        }

    # Submit to ComfyUI
    client_id = str(uuid.uuid4())
    payload = {"prompt": prompt_data, "client_id": client_id}
    r = requests.post(f"{server}/prompt", json=payload, timeout=120)

    if r.status_code != 200:
        print("❌ ComfyUI rejected the request:")
        print(r.text)
        return None

    data = r.json()
    prompt_id = data.get("prompt_id", client_id)
    print(f"✅ Queued prompt: {prompt_id}")

    # Poll until image is ready
    deadline = time.time() + 600
    while time.time() < deadline:
        hist = requests.get(f"{server}/history/{prompt_id}", timeout=60).json()
        item = hist.get(prompt_id)
        if item and "outputs" in item:
            for node_id, node_out in item["outputs"].items():
                for im in node_out.get("images", []):
                    fn = im.get("filename")
                    sub = im.get("subfolder", "")
                    typ = im.get("type", "output")
                    if fn:
                        out = download_image(fn, server, subfolder=sub, folder_type=typ)
                        print(f"✅ Saved result → {out}")
                        return out
        time.sleep(0.5)

    print("❌ Timeout: No output image found.")
    return None


# ------------------- Example Single Image Run -------------------
if __name__ == "__main__":
    image_path = "input/images/input_1 (1).png"   # 👈 your image file
    output_resized = "output/sample_result.png"

    positive_prompt = (
        "restore and colorize human skin with natural, realistic tones, "
        "clothes and dresses should remain as original, "
        
    )

    negative_prompt = (
        "colorful clothes"
    )

    result = run_prompt(
        PROMPT_JSON,
        image_path,
        positive_prompt,
        negative_prompt,
        seed=123456789,
        steps=5,
        cfg=1.5
    )
# "coloring clothes or dresses with any colors other than black"
    if result:
        orig_w, orig_h = Image.open(image_path).size
        out_img = Image.open(result).resize((orig_w, orig_h), Image.LANCZOS)
        os.makedirs(os.path.dirname(output_resized), exist_ok=True)
        out_img.save(output_resized)
        print(f"🎨 Final saved → {output_resized} ({orig_w}x{orig_h})")


✅ Uploaded: input_1 (1).png
✅ Queued prompt: 6f7c615a-a16b-4351-94d6-f1b118282b3c
✅ Saved result → results_qwen_api/ComfyUI_00049_.png
🎨 Final saved → output/sample_result.png (1923x841)


In [9]:
import os
import json
import uuid
import time
import requests
from PIL import Image
from tqdm import tqdm

COMFY = "http://127.0.0.1:8188"
PROMPT_JSON = "image_qwen_image_edit.json"
OUTPUT_DIR = "results_qwen_api"
os.makedirs(OUTPUT_DIR, exist_ok=True)


# --- Upload image to ComfyUI ---
def upload_image(image_path, server=COMFY):
    img_name = os.path.basename(image_path)
    with open(image_path, "rb") as f:
        files = {"image": (img_name, f, "image/png")}
        data = {"type": "input", "overwrite": "true"}
        r = requests.post(f"{server}/upload/image", files=files, data=data, timeout=120)
        r.raise_for_status()
    return img_name


# --- Download result image ---
def download_image(filename, server=COMFY, *, subfolder="", folder_type="output"):
    params = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    r = requests.get(f"{server}/view", params=params, timeout=120)
    r.raise_for_status()
    local_path = os.path.join(OUTPUT_DIR, filename)
    with open(local_path, "wb") as f:
        f.write(r.content)
    return local_path


# --- Run ComfyUI prompt with adjustable parameters (positive + negative prompts) ---
def run_prompt(
    prompt_json_path,
    image_path,
    positive_prompt,
    negative_prompt="",
    server=COMFY,
    seed=None,
    steps=None,
    cfg=None
):
    with open(prompt_json_path, "r") as f:
        prompt_data = json.load(f)

    # Upload image
    uploaded_img = upload_image(image_path, server)
    print(f"✅ Uploaded: {uploaded_img}")

    # Patch LoadImage node
    if "78" in prompt_data:
        prompt_data["78"]["inputs"]["image"] = uploaded_img

    # Positive prompt
    if "76" in prompt_data:
        prompt_data["76"]["inputs"]["prompt"] = positive_prompt

    # Negative prompt
    if "77" in prompt_data:
        prompt_data["77"]["inputs"]["prompt"] = negative_prompt

    # KSampler parameters
    if "3" in prompt_data:
        ks = prompt_data["3"]["inputs"]
        if seed is not None:
            ks["seed"] = int(seed)
        if steps is not None:
            ks["steps"] = int(steps)
        if cfg is not None:
            ks["cfg"] = float(cfg)

    # SaveImage node
    if "60" in prompt_data:
        prompt_data["60"]["inputs"] = {
            "filename_prefix": "ComfyUI",
            "images": ["8", 0]
        }

    # Submit prompt to ComfyUI server
    client_id = str(uuid.uuid4())
    payload = {"prompt": prompt_data, "client_id": client_id}
    r = requests.post(f"{server}/prompt", json=payload, timeout=120)

    if r.status_code != 200:
        print("❌ ComfyUI rejected the request:")
        print("Status:", r.status_code)
        print("Response:", r.text)
        return None

    data = r.json()
    prompt_id = data.get("prompt_id", client_id)
    print(f"✅ Queued prompt: {prompt_id}")

    # Poll for output
    deadline = time.time() + 600
    while time.time() < deadline:
        hist = requests.get(f"{server}/history/{prompt_id}", timeout=60).json()
        item = hist.get(prompt_id)
        if item and "outputs" in item:
            for node_id, node_out in item["outputs"].items():
                for im in node_out.get("images", []):
                    fn = im.get("filename")
                    sub = im.get("subfolder", "")
                    typ = im.get("type", "output")
                    if fn:
                        out = download_image(fn, server, subfolder=sub, folder_type=typ)
                        print(f"✅ Saved: {out}")
                        return out
        time.sleep(0.5)

    print("❌ Timeout: No output image found.")
    return None


# --- Batch processor for folders ---
def process_folder(
    input_folder,
    output_folder,
    positive_prompt,
    negative_prompt="",
    seed=None,
    steps=None,
    cfg=None
):
    os.makedirs(output_folder, exist_ok=True)
    image_files = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".png", ".jpg", ".jpeg"))
    ]

    print(f"\n📂 Found {len(image_files)} image(s) to process.\n")
    success_count, skip_count, fail_count = 0, 0, 0

    for fname in tqdm(image_files, desc="Processing images", ncols=100):
        in_path = os.path.join(input_folder, fname)
        out_path = os.path.join(output_folder, fname)

        if os.path.exists(out_path):
            skip_count += 1
            tqdm.write(f"⚙️  Skipping {fname}, already exists.")
            continue

        try:
            tqdm.write(f"🚀 Processing {fname} ...")
            result = run_prompt(
                PROMPT_JSON,
                in_path,
                positive_prompt,
                negative_prompt=negative_prompt,
                seed=seed,
                steps=steps,
                cfg=cfg
            )

            if result:
                orig_w, orig_h = Image.open(in_path).size
                out_img = Image.open(result)
                out_img = out_img.resize((orig_w, orig_h), Image.LANCZOS)
                out_img.save(out_path)
                success_count += 1
                tqdm.write(f"✅ Saved → {out_path} ({orig_w}x{orig_h})")
            else:
                fail_count += 1
                tqdm.write(f"⚠️ No output for {fname}")

        except Exception as e:
            fail_count += 1
            tqdm.write(f"❌ Error processing {fname}: {e}")

        time.sleep(1)

    print("\n📊 Summary:")
    print(f"✅ Success: {success_count}")
    print(f"⚙️  Skipped: {skip_count}")
    print(f"❌ Failed:  {fail_count}")
    print(f"📁 Output folder: {output_folder}")


# ------------------- Example Usage -------------------
if __name__ == "__main__":
    input_folder = "input/testing"
    output_folder = "output/qwen/prompt_8"
    positive_prompt = (
    "colorize only human skin"
    "keep all non-skin regions perfectly preserved"
    )

    negative_prompt = (
    " recolor clothes "
    )



    process_folder(
        input_folder,
        output_folder,
        positive_prompt,
        negative_prompt,
        seed=123456789,   # 👈 your seed
        steps=5,          # 👈 denoising steps
        cfg=1.5           # 👈 CFG guidance
    )



📂 Found 3 image(s) to process.



Processing images:   0%|                                                      | 0/3 [00:00<?, ?it/s]

🚀 Processing input_2 (1).png ...
✅ Uploaded: input_2 (1).png
✅ Queued prompt: 11d72d61-1ce1-4c6a-a242-1b6dc8468804
✅ Saved: results_qwen_api/ComfyUI_00026_.png


Processing images:   0%|                                                      | 0/3 [01:28<?, ?it/s]

✅ Saved → output/qwen/prompt_8/input_2 (1).png (1280x720)


...

🚀 Processing input_1 (1).png ...
✅ Uploaded: input_1 (1).png
✅ Queued prompt: 9376ff41-0c82-42f6-9dba-3072eb9c5fd9
✅ Saved: results_qwen_api/ComfyUI_00028_.png


Processing images:  67%|██████████████████████████████▋               | 2/3 [04:27<01:29, 89.68s/it]

✅ Saved → output/qwen/prompt_8/input_1 (1).png (1923x841)


Processing images: 100%|██████████████████████████████████████████████| 3/3 [04:28<00:00, 89.56s/it]


📊 Summary:
✅ Success: 3
⚙️  Skipped: 0
❌ Failed:  0
📁 Output folder: output/qwen/prompt_8


In [34]:
import os
import json
import uuid
import time
import requests
from PIL import Image

# ---------------- Configuration ----------------
COMFY = "http://127.0.0.1:8188"
PROMPT_JSON = "image_qwen_image_edit.json"
OUTPUT_DIR = "results_qwen_api"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ------------------------------------------------
def upload_image(image_path, server=COMFY):
    """Upload an image to ComfyUI server."""
    img_name = os.path.basename(image_path)
    with open(image_path, "rb") as f:
        files = {"image": (img_name, f, "image/png")}
        data = {"type": "input", "overwrite": "true"}
        r = requests.post(f"{server}/upload/image", files=files, data=data, timeout=120)
        r.raise_for_status()
    return img_name


def download_image(filename, server=COMFY, *, subfolder="", folder_type="output"):
    """Download generated image from ComfyUI output folder."""
    params = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    r = requests.get(f"{server}/view", params=params, timeout=120)
    r.raise_for_status()
    local_path = os.path.join(OUTPUT_DIR, filename)
    with open(local_path, "wb") as f:
        f.write(r.content)
    return local_path


def run_prompt(
    prompt_json_path,
    image_path,
    positive_prompt,
    negative_prompt="",
    server=COMFY,
    seed=None,
    steps=None,
    cfg=None
):
    """Run a single ComfyUI prompt with positive and negative text inputs."""
    with open(prompt_json_path, "r") as f:
        prompt_data = json.load(f)

    uploaded_img = upload_image(image_path, server)
    print(f"✅ Uploaded: {uploaded_img}")

    # LoadImage node
    if "78" in prompt_data:
        prompt_data["78"]["inputs"]["image"] = uploaded_img

    # Positive prompt
    if "76" in prompt_data:
        prompt_data["76"]["inputs"]["prompt"] = positive_prompt

    # Negative prompt
    if "77" in prompt_data:
        prompt_data["77"]["inputs"]["prompt"] = negative_prompt

    # Sampling options
    if "3" in prompt_data:
        ks = prompt_data["3"]["inputs"]
        if seed is not None:
            ks["seed"] = int(seed)
        if steps is not None:
            ks["steps"] = int(steps)
        if cfg is not None:
            ks["cfg"] = float(cfg)

    # SaveImage node fix
    if "60" in prompt_data:
        prompt_data["60"]["inputs"] = {
            "filename_prefix": "ComfyUI",
            "images": ["8", 0]
        }

    # Submit to ComfyUI
    client_id = str(uuid.uuid4())
    payload = {"prompt": prompt_data, "client_id": client_id}
    r = requests.post(f"{server}/prompt", json=payload, timeout=120)

    if r.status_code != 200:
        print("❌ ComfyUI rejected the request:")
        print(r.text)
        return None

    data = r.json()
    prompt_id = data.get("prompt_id", client_id)
    print(f"✅ Queued prompt: {prompt_id}")

    # Poll until image is ready
    deadline = time.time() + 600
    while time.time() < deadline:
        hist = requests.get(f"{server}/history/{prompt_id}", timeout=60).json()
        item = hist.get(prompt_id)
        if item and "outputs" in item:
            for node_id, node_out in item["outputs"].items():
                for im in node_out.get("images", []):
                    fn = im.get("filename")
                    sub = im.get("subfolder", "")
                    typ = im.get("type", "output")
                    if fn:
                        out = download_image(fn, server, subfolder=sub, folder_type=typ)
                        print(f"✅ Saved result → {out}")
                        return out
        time.sleep(0.5)

    print("❌ Timeout: No output image found.")
    return None


# ------------------- Example Single Image Run -------------------
if __name__ == "__main__":
    image_path = "input/images/input_1 (1).png"   # 👈 your image file
    output_resized = "output/prompt_1.png"

    positive_prompt = (
    "colorize only human skin"
    "keep all non-skin regions perfectly preserved"
    )

    negative_prompt = (
    " recolor clothes "
    )


   
    result = run_prompt(
        PROMPT_JSON,
        image_path,
        positive_prompt,
        negative_prompt,
        seed=123456789,
        steps=5,
        cfg=1.5
    )

    if result:
        orig_w, orig_h = Image.open(image_path).size
        out_img = Image.open(result).resize((orig_w, orig_h), Image.LANCZOS)
        os.makedirs(os.path.dirname(output_resized), exist_ok=True)
        out_img.save(output_resized)
        print(f"🎨 Final saved → {output_resized} ({orig_w}x{orig_h})")


✅ Uploaded: input_10 (1).png
✅ Queued prompt: 207bdc0e-09e7-48ac-95c3-85605570fc91
✅ Saved result → results_qwen_api/ComfyUI_00001_.png
🎨 Final saved → output/prompt_1.png (1280x720)
